In [117]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime

# Vis. modules.
import altair as alt


#### Reformat data into old Fitbit export style.



#### Sleep


In [2]:
filepath = r'~/Downloads/takeout-20240330T183803Z-001/Takeout/Fitbit/Global Export Data'
json_files = os.listdir(os.path.expanduser(filepath))
sleep_files = [file for file in json_files if re.match(r'sleep-\d{4}-\d{2}-\d{2}\.json$', file)]
sleep_files.sort()
sleep_files[:5] # Check first 5 are as expected.

['sleep-2016-08-29.json',
 'sleep-2016-09-28.json',
 'sleep-2016-10-28.json',
 'sleep-2016-11-27.json',
 'sleep-2016-12-27.json']

In [3]:
dfs = []
for file in sleep_files:
    file_path = os.path.join(filepath + '/', file)
    df = pd.read_json(file_path)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

In [4]:
unnested = pd.json_normalize(df['levels'])

In [5]:
df = pd.merge(
    left=df[['startTime', 'endTime', 'minutesAsleep', 'minutesAwake', 'timeInBed']], 
    right=unnested.iloc[:, 2:], 
    left_index=True, 
    right_index=True
)

In [6]:
df.rename(columns={
    'summary.wake.count':'Number of Awakenings',
    'summary.rem.minutes': 'Minutes REM Sleep',
    'summary.light.minutes':'Minutes Light Sleep',
    'summary.deep.minutes':'Minutes Deep Sleep',
}, inplace=True)

In [7]:
tmp_col = df['Number of Awakenings'].copy()
df.drop(columns=['Number of Awakenings'], inplace=True)
df.insert(4, 'Number of Awakenings', tmp_col)

In [8]:
df = df[[
    'startTime', 
    'endTime', 
    'minutesAsleep', 
    'minutesAwake', 
    'Number of Awakenings', 
    'timeInBed', 
    'Minutes REM Sleep',
    'Minutes Light Sleep',
    'Minutes Deep Sleep',
]]

In [9]:
df = df.astype({'startTime':'datetime64[ns]', 'endTime':'datetime64[ns]'})

In [10]:
def convert_to_fitbit_time(cols):
    """Converts millisecond UTC timestamp into native 12h time."""
    for col in cols:
        df[col] = df[col].apply(lambda x: x.strftime('%Y-%m-%d %I:%M%p'))
        df[col] = df[col].apply(lambda x: x.replace(' 0', ' '))

        
convert_to_fitbit_time(['startTime', 'endTime'])

In [11]:
df.head()

,startTime,endTime,minutesAsleep,minutesAwake,Number of Awakenings,timeInBed,Minutes REM Sleep,Minutes Light Sleep,Minutes Deep Sleep
0,2016-09-27 10:21PM,2016-09-28 7:33AM,521,26,NaN,551,NaN,NaN,NaN
1,2016-09-26 11:54PM,2016-09-27 7:34AM,439,21,NaN,460,NaN,NaN,NaN
2,2016-09-26 12:14AM,2016-09-26 7:24AM,403,27,NaN,430,NaN,NaN,NaN
3,2016-09-24 10:41PM,2016-09-25 9:28AM,599,47,NaN,647,NaN,NaN,NaN
4,2016-09-23 11:07PM,2016-09-24 5:22AM,357,17,NaN,374,NaN,NaN,NaN


In [12]:
# Outputs FitBit sleep data in old export style wherever your Jupyter Notebook is located.
df.to_csv('./fitbit_sleep_data_in_old_export_format.csv', index=False, header=False, na_rep='N/A')


#### Body


-  bodyweight
-  BMI
-  Fat

In [37]:
weight_files = [file for file in json_files if re.match(r'weight-\d{4}-\d{2}-\d{2}\.json$', file)]
weight_files.sort()
weight_files[:5] # Check first 5 are as expected.

['weight-2016-08-29.json',
 'weight-2016-12-27.json',
 'weight-2017-01-26.json',
 'weight-2017-02-25.json',
 'weight-2017-06-25.json']

In [38]:
dfs = []
for file in weight_files:
    file_path = os.path.join(filepath + '/', file)
    df = pd.read_json(file_path)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

In [39]:
df = df.rename(
    columns={'weight':'Bodyweight', 'bmi':'BMI', 'fat':'Fat'}
).drop(
    columns=['logId', 'source']
).reindex(
    columns=['date', 'time', 'Bodyweight', 'BMI', 'Fat']
)

In [40]:
df

,date,time,Bodyweight,BMI,Fat
0,2016-08-30,23:59:59,212.0,29.58,NaN
1,2017-01-01,18:25:24,212.1,29.60,29.879999
2,2017-01-01,18:59:52,211.6,29.53,29.378000
3,2017-01-01,23:59:59,211.5,29.52,NaN
4,2017-01-02,10:35:10,206.8,28.86,28.540001
...,...,...,...,...,...
108,2023-01-07,14:36:55,242.6,33.86,29.719999
109,2023-01-08,16:33:43,245.1,34.20,33.070000
110,2023-03-09,21:36:18,241.1,33.65,NaN
111,2023-12-08,11:47:54,239.5,33.43,32.458000



#### Activity



-  cal_burn
-  steps
-  distance
-  floors
-  mins_sedentary
-  mins_lightactive
-  mins_fairlyactive
-  mins_veryactive
-  cal_activity


In [46]:
calories_files = [file for file in json_files if re.match(r'calories-\d{4}-\d{2}-\d{2}\.json$', file)]
calories_files.sort()
calories_files[:5] # Check first 5 are as expected.

['calories-2016-08-29.json',
 'calories-2016-09-28.json',
 'calories-2016-10-28.json',
 'calories-2016-11-27.json',
 'calories-2016-12-27.json']

In [47]:
# Takes too long since there is 1 .json object for every mintue - should be lazily evaluated and aggregated daily.
dfs = []
for file in calories_files:
    file_path = os.path.join(filepath + '/', file)
    df = pd.read_json(file_path)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

In [48]:
df

,dateTime,value
0,2016-08-29 00:00:00,1.35
1,2016-08-29 00:01:00,1.35
2,2016-08-29 00:02:00,1.35
3,2016-08-29 00:03:00,1.35
4,2016-08-29 00:04:00,1.35
...,...,...
3989696,2024-03-30 14:56:00,1.41
3989697,2024-03-30 14:57:00,1.41
3989698,2024-03-30 14:58:00,1.41
3989699,2024-03-30 14:59:00,1.41



#### Resting Heart Rate (RHR)


In [49]:
# Functionalize this.
rhr_files = [file for file in json_files if re.match(r'resting_heart_rate-\d{4}-\d{2}-\d{2}\.json$', file)]
rhr_files.sort()
rhr_files[:5] # Check first 5 are as expected.

['resting_heart_rate-2016-08-29.json',
 'resting_heart_rate-2017-08-29.json',
 'resting_heart_rate-2018-08-29.json',
 'resting_heart_rate-2019-08-29.json',
 'resting_heart_rate-2020-08-28.json']

In [50]:
dfs = []
for file in rhr_files:
    file_path = os.path.join(filepath + '/', file)
    df = pd.read_json(file_path)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

In [121]:
unnested = pd.json_normalize(df['value'])
unnested['date'] = unnested['date'].replace({None: np.nan})
unnested['date'] = pd.to_datetime(unnested['date'], format='%m/%d/%y')
unnested = unnested.dropna(subset='date')
unnested.sample(5)

,date,value,error
1992,2022-02-11,70.424182,6.787087
1638,2021-02-22,65.628735,6.787087
1595,2021-01-10,67.416051,6.787087
2580,2023-09-22,68.201071,6.787736
1922,2021-12-03,65.439914,6.787087


In [151]:
alt.Chart(unnested).mark_line().encode(
    x=alt.X(
        'date', 
        axis=alt.Axis(domainOpacity=0, format='%b %y', grid=False)
    ),
    y=alt.Y(
        'value',
        axis=alt.Axis(title='RHR'),
        scale=alt.Scale(
            domain=[
                unnested['value'].min() - 3, unnested['value'].max() + 3]
        )
    )
).properties(width=800, title='Resting Heart Rate: Daily')

alt.Chart(...)